# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `clanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g., copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace.
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refer to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).


We here aim at presenting how template libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

On Windows OSes, the visual studio version used to compile future wrappers must be given.
But if the **SCons** tool is used, this version is known.

* to import **subprocess**.

In [2]:
import subprocess

* to detect the **Git** repository root

In [3]:
import os
GIT_ROOT = subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode()
GIT_ROOT = GIT_ROOT.replace('/', os.sep).strip()
GIT_ROOT = os.path.join(GIT_ROOT, 'share', 'git', 'STL')
from devops_tools import describe
os.environ['GIT_DESCRIBE_VERSION'] = describe.git_describe_version(GIT_ROOT)
os.environ['GIT_DESCRIBE_NUMBER'] = describe.git_describe_number(GIT_ROOT)
os.environ['DATETIME_DESCRIBE_VERSION'] = describe.datetime_describe_version(GIT_ROOT)
os.environ['DATETIME_DESCRIBE_NUMBER'] = describe.datetime_describe_number(GIT_ROOT)

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [4]:
SCONSCRIPT = os.path.join(GIT_ROOT, 'src', 'cpp', 'SConscript')
!pygmentize {SCONSCRIPT}

# -*-python-*-

VECTORS = ['Index',
           'int',
           'double',
           'std::string']

SETS = { 'less': ['Index',
                  'int',
                  'double',
                  'std::string'],
         'none': ['Index']}

HEADER = """\
#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
   

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [5]:
AUTOWIG = os.path.join(GIT_ROOT, 'src', 'cpp', 'AutoWIG.py')
!pygmentize {AUTOWIG}

def controller(asg, library=True, **kwargs):
    import autowig
    from autowig.asg import TemplateSpecializationProxy
    autowig.controller.plugin = 'default'
    asg = autowig.controller(asg, clean=False, **kwargs)
    if library:
        for function in asg['::statiskit::stl'].functions():
            if function.localname in ['generator', 'insert']:
                parameter = function.parameters[0].qualified_type.desugared_type
                if parameter.is_class:
                    function.parent = parameter.unqualified_type
        for mtd in asg['::statiskit::stl::String'].qualified_type.desugared_type.unqualified_type.methods():
            mtd.pybind11_export = False
    if 'class ::std::vector' in asg:
        for cls in asg['class ::std::vector'].specializations(partial=False):
            for method in cls.methods():
                if method.localname in ['resize', 'shrink_to_fit', 'operator[]']:
                    if isinstance(method.pybind11_export, bool):
     

Then, in addition to the **STL** library, the **StatisKit.STL** library has to be installed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [6]:
subprocess.call('conda remove libstatiskit_stl -y', shell=True)
SCONSIGN = os.path.join(GIT_ROOT, '.sconsign.dblite')
if is_windows:
    subprocess.call('del ' + SCONSIGN, shell=True)
else:
    subprocess.call('rm ' + SCONSIGN, shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'etc', 'conda', 'libstatiskit_stl')
subprocess.check_call('conda build ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y libstatiskit_stl --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

0

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [7]:
TYPEDEFS = os.path.join(GIT_ROOT, 'src', 'cpp', 'STL.h')
!pygmentize {TYPEDEFS}

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef size_t Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generator(const T&

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [8]:
if is_windows:
    subprocess.call('del ' + SCONSIGN, shell=True)
else:
    subprocess.call('rm ' + SCONSIGN, shell=True)
subprocess.check_call('scons cpp -C ' + GIT_ROOT, shell=True)

0

Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [9]:
if is_windows:
    subprocess.call('del ' + SCONSIGN, shell=True)
else:
    subprocess.call('rm ' + SCONSIGN, shell=True)
subprocess.check_call('scons autowig -C ' + GIT_ROOT, shell=True)

0

Here is the list of the generated wrappers (untracked files).

In [10]:
!git -C {GIT_ROOT} status

On branch fp17
Your branch is up to date with 'origin/fp17'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/cpp/STL.cpp
	modified:   src/cpp/STL.h

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/py/statiskit/stl/_stl.py
	src/py/wrapper/_stl.cpp
	src/py/wrapper/_stl.h
	src/py/wrapper/wrapper_107131f9768c56e794a9b0de728d1738.cpp
	src/py/wrapper/wrapper_10b14312eeb655268489cd34090870cf.cpp
	src/py/wrapper/wrapper_3b59a0980c80518c808634f7a84dc3cd.cpp
	src/py/wrapper/wrapper_448c20257e485acda59dc59305fceb58.cpp
	src/py/wrapper/wrapper_476c1c1f206251dba7af53c48f3f6e42.cpp
	src/py/wrapper/wrapper_6436891c9b6854f494789a812891cbe5.cpp
	src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
	src/py/wrapper/wrapper_745e4933f5b250d9bc5232fe864d0cf7.cpp
	src/py/wrapper/wrapper_74bc331707c1505eb2f807cae7c32bbb.cpp
	s

And here, we present the wrappers generated for the `std::vector< int >` class.

In [11]:
WRAPPER = os.path.join(GIT_ROOT, 'src', 'py', 'wrapper',
                       'wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp')
!pygmentize {WRAPPER}

#include "_stl.h"

void  (::std::vector< int, ::std::allocator< int > >::*method_pointer_3ee60599950b5555a32f72572e8ff771)(::std::vector< int, class ::std::allocator< int > >::size_type , ::std::vector< int, class ::std::allocator< int > >::value_type const &)= &::std::vector< int, class ::std::allocator< int > >::assign;
::std::vector< int, class ::std::allocator< int > >::size_type  (::std::vector< int, ::std::allocator< int > >::*method_pointer_2f0bd94041965427ab114d1ec9369eb1)()const= &::std::vector< int, class ::std::allocator< int > >::size;
::std::vector< int, class ::std::allocator< int > >::size_type  (::std::vector< int, ::std::allocator< int > >::*method_pointer_03cb2a43c5ae5df48ecc631a008fa511)()const= &::std::vector< int, class ::std::allocator< int > >::max_size;
::std::vector< int, class ::std::allocator< int > >::size_type  (::std::vector< int, ::std::allocator< int > >::*method_pointer_2d96cb90afc35aaaa142783706900e63)()const= &::std::vector< int, class ::std::allocato

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [12]:
subprocess.call('conda remove python-statiskit_stl -y', shell=True)
if is_windows:
    subprocess.call('del ' + SCONSIGN, shell=True)
else:
    subprocess.call('rm ' + SCONSIGN, shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'etc', 'conda', 'python-statiskit_stl')
subprocess.check_call('conda build ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y python-statiskit_stl --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

0

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [13]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

(-1, 0, 1)

In [14]:
list(v)

[-1, 0, 1]

In [15]:
v[0]

-1

In [16]:
v[0] = -2
v[0]

-2

Here is a report concerning objects wrapped using this notebook.

In [17]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SITE_SCONS'],
                       'site_autowig',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)

Headers: 104 (52580 SLOC)
Enumerations: 2 (0 wrapped)
Variables: 267 (0 wrapped)
Functions: 3825 (0 wrapped)
Classes: 4494 (0 wrapped)
